# 블루보틀 리뷰 분류하기

In [1]:
import pandas as pd

In [27]:
df = pd.read_csv('source/Bluebottle_review/data.csv', index_col=0)
df.head()

,cafe,review,score
0,블루보틀 역삼점,불친절해요. 배가 불렀는듯,2점
1,블루보틀 역삼점,아메리카노도 맛있고 놀라플로트도 맛있어요!! 사람만 안많으면 자주오고싶어요,5점
2,블루보틀 역삼점,좋습니다,5점
3,블루보틀 역삼점,쏘쏘. 그냥 테이크아웃하기 좋음,3점
4,블루보틀 역삼점,편히 머물 수 있는 공간은 아닌. 커피는 우유들어간건 맛있어요.. 디저트는 메종엠오,2점


In [9]:
df.shape

(204, 3)

In [10]:
df.isnull().sum()

cafe      0
review    0
score     0
dtype: int64

In [12]:
df.score.value_counts().sort_index()

1점    62
2점    34
3점    44
4점    27
5점    37
Name: score, dtype: int64

In [32]:
df['score'] = df['score'].apply(lambda x: '1점' if x == '2점' else x)

In [33]:
df['score'] = df['score'].apply(lambda x: 1 if x == '1점' else 0)

In [34]:
df

,cafe,review,score
0,블루보틀 역삼점,불친절해요. 배가 불렀는듯,1
1,블루보틀 역삼점,아메리카노도 맛있고 놀라플로트도 맛있어요!! 사람만 안많으면 자주오고싶어요,0
2,블루보틀 역삼점,좋습니다,0
3,블루보틀 역삼점,쏘쏘. 그냥 테이크아웃하기 좋음,0
4,블루보틀 역삼점,편히 머물 수 있는 공간은 아닌. 커피는 우유들어간건 맛있어요.. 디저트는 메종엠오,1
...,...,...,...
199,블루보틀 삼청한옥,사람들이 다 한방향으로 달려가길래 나는 무슨 연예인이 온 줄 알았다 이 땡볕에도 간...,0
200,블루보틀 삼청한옥,나가지도 않았는데 잔부터 치워버리는건 무슨 생각이지...? 옆에 짐도 버젓히 놔뒀는...,1
201,블루보틀 삼청한옥,최고예요,0
202,블루보틀 삼청점,내 입맛은 뉴요커가 아닌가봐요,1


In [37]:
from konlpy.tag import Okt
okt = Okt()

In [42]:
from tqdm import tqdm_notebook

In [49]:
words = []
for sentence in tqdm_notebook(df.review):
    w = okt.pos(sentence)
    for word, tag in w:
        if tag not in ('Josa', 'Punctuation'):
            words.append(word)

C:\Users\samsung\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [74]:
word = words.copy()

In [75]:
from collections import Counter
c = Counter(word).most_common()

In [76]:
word = []
for w, i in tqdm(c):
    if i > 4 :
        word.append(w) 

In [123]:
print(len(word))
word

78


['맛',
 '커피',
 '라떼',
 '가격',
 '너무',
 '줄',
 '직원',
 '보틀',
 '블루',
 '사람',
 '그냥',
 '생각',
 '인테리어',
 '보통',
 '정도',
 '별로',
 '원두',
 '서서',
 '매장',
 '매우',
 '해서',
 '층',
 '카페',
 '시간',
 '주문',
 '조금',
 '뉴올리언스',
 '하나',
 '굳이',
 '적',
 '공간',
 '최고',
 '합니다',
 '다른',
 '없음',
 '못',
 '한국',
 '인스타',
 '왜',
 '한번',
 '분들',
 '서비스',
 '좀',
 '별',
 '역시',
 '많이',
 '좋아요',
 '대기',
 '없이',
 '느낌',
 '음료',
 '맛있어요',
 '없고',
 '잘',
 '대비',
 '좌석',
 '스벅',
 '친절',
 '방문',
 '처음',
 '여기',
 '최악',
 '건물',
 '자리',
 '아닌',
 '우유',
 '적고',
 '마실',
 '아님',
 '콜드',
 '브루',
 '하지만',
 '분위기',
 '일행',
 '많아서',
 '절대',
 '있고',
 '진짜']

In [89]:
word.remove('들')

In [55]:
len(words)

3361

In [152]:
len(word)

78

In [153]:
vec = []
for i in range(len(word)):
    vec.append([])

In [154]:
for text in tqdm(df.review):
    t = okt.morphs(text)
    for i, w in enumerate(word):
        vec[i].append(t.count(w))

In [155]:
df_idf = pd.DataFrame(vec).T
df_idf

,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,77
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,0,0,0,0,0,2,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
200,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [156]:
df_idf.columns = word

In [157]:
df_idf.head()

,맛,커피,라떼,가격,너무,줄,직원,보틀,블루,사람,...,아님,콜드,브루,하지만,분위기,일행,많아서,절대,있고,진짜
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [158]:
df_idf.shape

(204, 78)

In [159]:
df_idf['bad'] = df.score

In [160]:
df_idf

,맛,커피,라떼,가격,너무,줄,직원,보틀,블루,사람,...,콜드,브루,하지만,분위기,일행,많아서,절대,있고,진짜,bad
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,0,0,0,0,0,2,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
200,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
201,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [162]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_idf.drop('bad', axis=1), df_idf['bad'])

In [163]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

C:\Users\samsung\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [168]:
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [170]:
y_pred = logreg.predict(X_train)
print(f1_score(y_train, y_pred))
confusion_matrix(y_train, y_pred)

0.8346456692913385


array([[79,  7],
       [14, 53]], dtype=int64)

In [171]:
y_pred = logreg.predict(X_test)
print(f1_score(y_test, y_pred))
confusion_matrix(y_test, y_pred)

0.6956521739130435


array([[21,  1],
       [13, 16]], dtype=int64)